<a href="https://colab.research.google.com/github/Himanshu069/trictrl/blob/sim/Data_Generation_NMPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Triangle-Approximate NMPC Dataset Generation.ipynb
Automatically generated by Colab.
"""

!pip install do-mpc
!pip install numpy
!pip install casadi
!pip install matplotlib
!pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 MB 12.3 MB/s eta 0:00:00


In [ ]:

import numpy as np
import pandas as pd
import sys
from casadi import *
import do_mpc
import matplotlib.pyplot as plt

# ----------------------------
# Model setup
# ----------------------------
model_type = 'continuous'  # 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)



/usr/local/lib/python3.12/dist-packages/do_mpc/sysid/__init__.py:15: UserWarning: The ONNX feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The ONNX feature is not available. Please install the full version of do-mpc to access this feature.')
/usr/local/lib/python3.12/dist-packages/do_mpc/opcua/__init__.py:14: UserWarning: The opcua feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The opcua feature is not available. Please install the full version of do-mpc to access this feature.')


In [ ]:
# System parameters
M = 0.01825
mb = 0.286
mw = 0.079
Iw = 0.000040394
Ib = 0.0004370975
lb = 0.05
lw = 0.05
g = 9.81
I = Ib + mb*lb*lb + mw*lw*lw

t_abs = model.set_variable(var_type='_tvp', var_name='t_abs')
# States
theta = model.set_variable('_x','theta')
phi = model.set_variable('_x','phi')
dtheta = model.set_variable('_x','dtheta')
dphi = model.set_variable('_x','dphi')

# Control
u = model.set_variable('_u','torque')



In [ ]:
# Algebraic variables
ddtheta = model.set_variable('_z', 'ddtheta')
ddphi = model.set_variable('_z', 'ddphi')



In [ ]:
# RHS
model.set_rhs('theta', dtheta)
model.set_rhs('phi', dphi)
model.set_rhs('dtheta', ddtheta)
model.set_rhs('dphi', ddphi)



In [ ]:
# Euler-Lagrange equations
euler_lagrange = vertcat(
    (I+Iw)*ddtheta + Iw*ddphi - M*g*sin(theta),
    Iw*(ddtheta + ddphi) - u
)
model.set_alg('euler_lagrange', euler_lagrange)


In [ ]:
model.setup()



In [ ]:
# ----------------------------
# NMPC controller setup
# ----------------------------
mpc = do_mpc.controller.MPC(model)


In [ ]:
setup_mpc = {
    'n_horizon': 50,
    'n_robust': 0,
    'open_loop': 0,
    't_step': 0.04,
    'state_discretization': 'collocation',
    'collocation_type': 'radau',
    'collocation_deg': 6,
    'collocation_ni': 1,
    'store_full_solution': True,
    'nlpsol_opts': {
        'ipopt.linear_solver': 'mumps',
      # suppress CasADi-level output
    }
}
mpc.set_param(**setup_mpc)



In [ ]:
# Objective
Q1 = 1000
Q2 = 5
Q3 = 1.0
R = 1.0
omega_s = 0
omega_ref = omega_s + 24.76 * sin(78.54 * model.tvp['t_abs']-1.57)
mterm = Q1*model.x['theta']**2 + Q2*model.x['dtheta']**2 + Q3*(model.x['dphi']-omega_s)**2
lterm = Q1*model.x['theta']**2 + Q2*model.x['dtheta']**2 + Q3*(model.x['dphi']-omega_s)**2
mpc.set_objective(mterm=mterm, lterm=lterm)
mpc.set_rterm(torque= R)

mpc.bounds['lower','_u','torque'] = -0.5
mpc.bounds['upper','_u','torque'] = 0.5
# mpc.bounds['lower','_x','dphi'] = -1000
# mpc.bounds['upper','_x','dphi'] = 1000


In [ ]:
tvp_template_mpc = mpc.get_tvp_template()
t_step = 0.04
def tvp_fun(t_now):
        for k in range(50+1):
                tvp_template_mpc['_tvp',k,'t_abs'] = t_now + k*t_step

        return tvp_template_mpc

mpc.set_tvp_fun(tvp_fun)

In [ ]:

mpc.setup()

# ----------------------------
# Estimator and simulator
# ----------------------------
estimator = do_mpc.estimator.StateFeedback(model)
simulator = do_mpc.simulator.Simulator(model)
params_simulator = {
    'integration_tool': 'idas',
    'abstol': 1e-8,
    'reltol': 1e-8,
    't_step': 0.04
}
simulator.set_param(**params_simulator)
tvp_template_sim = simulator.get_tvp_template()
def tvp_fun_sim(t_now):
        tvp_template_sim['t_abs'] = t_now
        return tvp_template_sim
simulator.set_tvp_fun(tvp_fun_sim)
simulator.setup()


In [ ]:
x0 = np.array([[3.14, 0, 0, 0]]).T

simulator.x0 = x0
mpc.x0 = x0
estimator.x0 = x0
mpc.reset_history()
u_step = mpc.make_step(x0)
print("u0", u_step)

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:    12504
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      552

Total number of variables............................:     2382
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       50
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2304
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.0882599e+06 1.00e-02 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define path inside your Drive (you can change folder path)
save_path = '/content/drive/MyDrive/nmpc_dataset[tv-10%].csv'

n_theta = 100
theta_pos = np.linspace(np.pi/n_theta, np.pi, n_theta)

theta_neg = -theta_pos[::-1]

theta_values = np.concatenate([theta_neg, theta_pos])
n_steps = 50
t_step = params_simulator['t_step']

dataset = []
for i, theta0 in enumerate(theta_values):
    x0 = np.array([[theta0, 0, 0, 0]]).T

    simulator.x0 = x0
    mpc.x0 = x0
    estimator.x0 = x0
    mpc.reset_history()

    t = 0.0
    for k in range(n_steps):
        u_step = mpc.make_step(x0)

        # tvp_values = mpc.data['_tvp']
        y_next = simulator.make_step(u_step)
        x_next = estimator.make_step(y_next)

        dataset.append([t, x0[0,0], x0[1,0], x0[2,0], x0[3,0], u_step[0,0]])

        # Print inside the loop every 5 timesteps
        if k % 5 == 0:
            print(f"Traj {i+1}/{len(theta_values)}, step {k+1}/{n_steps}, t={t:.2f}, theta={x0[0,0]:.3f}, u={u_step[0,0]:.3f}, dphi={x0[3,0]:.1f}")
        x0 = x_next
        t += t_step
        # t_global += t_step

# Save dataset to Drive
columns = ['time', 'theta', 'phi', 'dtheta', 'dphi', 'u']
df = pd.DataFrame(dataset, columns=columns)
df.to_csv(save_path, index=False)

# Print after saving
print(f"\nDataset generation complete. Total samples: {len(dataset)}")
print(f"Dataset saved to Google Drive as '{save_path}'")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/do_mpc/controller/_mpc.py:1004: UserWarning: Intial guess for the MPC was not set. The solver call is likely to fail.
  warnings.warn('Intial guess for the MPC was not set. The solver call is likely to fail.')


Traj 1/200, step 1/50, t=0.00, theta=-3.142, u=-0.500, dphi=0.0
Traj 1/200, step 6/50, t=0.20, theta=0.221, u=0.149, dphi=-528.6
Traj 1/200, step 11/50, t=0.40, theta=0.197, u=0.071, dphi=-85.7
Traj 1/200, step 16/50, t=0.60, theta=0.039, u=-0.046, dphi=18.2
Traj 1/200, step 21/50, t=0.80, theta=0.006, u=0.049, dphi=-25.3
Traj 1/200, step 26/50, t=1.00, theta=0.001, u=-0.048, dphi=24.6
Traj 1/200, step 31/50, t=1.20, theta=0.000, u=0.048, dphi=-24.7
Traj 1/200, step 36/50, t=1.40, theta=0.000, u=-0.048, dphi=24.7
Traj 1/200, step 41/50, t=1.60, theta=0.000, u=0.048, dphi=-24.7
Traj 1/200, step 46/50, t=1.80, theta=0.000, u=-0.048, dphi=24.7
Traj 2/200, step 1/50, t=0.00, theta=-3.110, u=-0.500, dphi=0.0
Traj 2/200, step 6/50, t=0.20, theta=0.230, u=0.149, dphi=-530.5
Traj 2/200, step 11/50, t=0.40, theta=0.198, u=0.071, dphi=-86.1
Traj 2/200, step 16/50, t=0.60, theta=0.039, u=-0.046, dphi=18.2
Traj 2/200, step 21/50, t=0.80, theta=0.006, u=0.049, dphi=-25.3
Traj 2/200, step 26/50, t=1

In [ ]:
!cat nmpc_dataset.csv

cat: nmpc_dataset.csv: No such file or directory
